In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from src.mimic_fxns import (connect, insert_data, data_extraction, transform_labs, hot_coding, age_bands, 
                            month_transform, data_processing, normal_lab_vital_ranges, 
                            data_processing_column_refs)
import pickle


In [2]:
#load constants and connect to database
conn = connect()
normal_ranges = normal_lab_vital_ranges()
id_cols, month_col, age_col, encoding_cols, chronic_cols, merge_cols = data_processing_column_refs()

Connecting to PostgreSQL database...
Connection successful


In [3]:
print('Data loading...')
labpath = './src/bulk_member_model_labs.sql'
pt_labs = data_extraction(labpath, conn)
pt_labs.dropna(axis=0, subset=['chartdate'], inplace=True)
pt_labs.reset_index(drop=True)
pt_labs['chartdate'] = pd.to_datetime(pt_labs['chartdate'])
print('Lab data loaded...')

patientpath = './src/bulk_member_model_extraction.sql'
pt_admit = data_extraction(patientpath, conn)
pt_admit.dropna(axis=0, subset = ['intime', 'outtime'], inplace=True)
pt_admit.reset_index(drop=True)

print('Patient admitting data loaded...')

vitalspath = './src/bulk_member_model_chart_events.sql'
pt_vitals = data_extraction(vitalspath, conn)
pt_vitals.dropna(axis=0, subset=['chartdate'], inplace=True)
pt_vitals.reset_index(drop=True)
pt_vitals['chartdate'] = pd.to_datetime(pt_vitals['chartdate'])
print('Vitals data loaded...')

echoecgpath = './src/bulk_member_echo_ecg_notes.sql'
pt_ee_notes = data_extraction(echoecgpath, conn)
pt_ee_notes.dropna(axis=0, inplace=True)
groupcols = id_cols + ['icustay_id','chartdate','echo_ecg']
pt_ee_docs = pt_ee_notes[groupcols].groupby(id_cols+['icustay_id', 'chartdate']).sum()
pt_ee_docs.reset_index(inplace=True)
pt_ee_docs['chartdate'] = pd.to_datetime(pt_ee_docs['chartdate'])

print('Echo and ECG data loaded...')


Data loading...
Lab data loaded...
Patient admitting data loaded...
Vitals data loaded...
Echo and ECG data loaded...


In [4]:
#constant creation and instantiation of bucket for creating daily rows for each admission
id_cols = ['subject_id', 'hadm_id', 'icustay_id']
day_cols = ['dos', 'day_n']
left_cols = ['subject_id', 'hadm_id', 'icustay_id', 'dos']
right_cols = ['subject_id', 'hadm_id', 'icustay_id', 'chartdate']
top_merge_cols = merge_cols.copy()
top_merge_cols.append('icustay_id')
dailies = []

In [5]:
for idx, admission in enumerate(pt_admit['icustay_id']):
    adm_data = pt_admit[pt_admit['icustay_id'] == admission]
    intime = pd.DataFrame(adm_data['intime'])
    outtime = pd.DataFrame(adm_data['outtime'])
    dates_of_service = pd.date_range(intime.iloc[0,0], outtime.iloc[0,0],freq='D')
    days = list(range(1,len(dates_of_service)+1))
    daily = pd.DataFrame(data = zip(dates_of_service, days), index = None, columns=day_cols)
    for col in id_cols:
        daily[col] = pd.DataFrame(adm_data[col]).iloc[0,0]
    dailies.append(daily)
    if idx % 10000 == 0:
        print (f'{idx} * 1,000 admissions processed')

daily_df = pd.concat(dailies)
daily_df['stay_id'] = daily_df['subject_id'].astype('str') + '_' + daily_df['hadm_id'].astype('str') + '_' + daily_df['icustay_id'].astype('str')

In [6]:
daily_df = daily_df.merge(pt_labs, how = 'left', left_on=left_cols, right_on=right_cols)
daily_df.drop('chartdate', axis=1, inplace=True)
daily_df = daily_df.merge(pt_vitals, how = 'left', left_on=left_cols, right_on=right_cols)
daily_df.drop('chartdate', axis=1, inplace=True)
daily_df = daily_df.merge(pt_ee_docs, how = 'left', left_on=left_cols, right_on=right_cols)
daily_df.drop('chartdate', axis=1, inplace=True)

In [7]:
icustays = daily_df['icustay_id'].unique()
stays = []
for stay in icustays:
    data = daily_df[daily_df['icustay_id'] == stay].copy()
    data.fillna(method='ffill', inplace=True)
    stays.append(data)
modified = pd.concat(stays)

In [8]:
daily_df = modified.copy()

In [9]:
daily_df = daily_df.merge(pt_admit, how = 'inner', on = top_merge_cols)

In [10]:

ee = list(pt_ee_notes.columns)
ee.remove('chartdate')
ee.append('dos')
ad = list(pt_admit.columns)
ad.remove('intime')
ad.remove('outtime')
ad.append('dos')
ad.append('stay_id')
lb = list(pt_labs.columns)
lb.remove('chartdate')
lb.append('dos')
vt = list(pt_vitals.columns)
vt.remove('chartdate')
vt.append('dos')


labs = daily_df[lb].copy()
admits = daily_df[ad].copy()
vitals = daily_df[vt].copy()
echoecg = daily_df[ee].copy()
echoecg.dropna(axis=0,how='any', inplace=True)
merge_cols.append('dos')

In [11]:
X,y = data_processing(labs, normal_ranges, admits, month_col, encoding_cols, age_col, 
                          chronic_cols, merge_cols, [], vitals, echoecg)

post chronic (160101, 44)
(160101, 21) post labs (160101, 62)
(29215, 6) post echo (160101, 65)


In [12]:
X.head()

,subject_id,hadm_id,icustay_id_x,gender,readmit_thirty,dos,stay_id,admit_month_transform,x0_ELECTIVE,x0_EMERGENCY,...,bun,wbc,icustay_id_y,temperature,heartrate,systolic_bp,mean_arterial_pressure,icustay_id_y,top2,top1
0,72707,192735,265357,0,0,2181-09-13,72707_192735_265357,8.034845e-02,0.0,1.0,...,0.000000,0.000000,265357,-0.105625,0.0196,0.090000,0.00,NaN,-1.0,-1.0
1,72707,192735,265357,0,0,2181-09-14,72707_192735_265357,8.034845e-02,0.0,1.0,...,0.000000,0.000000,265357,0.000000,0.5776,0.134444,0.00,265357.0,7.0,2.0
2,72555,194577,291971,1,0,2144-08-18,72555_194577_291971,-3.061617e-17,0.0,1.0,...,0.000000,0.000000,291971,1.500625,0.0400,0.490000,0.00,NaN,-1.0,-1.0
3,72555,194577,291971,1,0,2144-08-19,72555_194577_291971,-3.061617e-17,0.0,1.0,...,0.000000,0.340278,291971,0.275625,0.0256,0.111111,0.00,NaN,-1.0,-1.0
4,91819,184658,299557,1,0,2178-06-23,91819_184658_299557,-1.231010e-01,0.0,1.0,...,0.444444,0.043403,299557,0.111111,0.0000,0.000000,0.16,NaN,-1.0,-1.0


In [13]:
X_predict = X.drop(['subject_id', 'hadm_id', 'icustay_id_x', 'dos', 'stay_id', 'icustay_id_y', 'icustay_id_y'], axis=1)

In [14]:
X = X.drop(['icustay_id_y', 'icustay_id_y'], axis=1)

In [18]:
with open('./src/icu_modelrf.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    cm_rf = pickle.load(f)
    

In [145]:
X_predict.shape

(160101, 43)

In [146]:
cm_rf.feature_importances_.shape

(43,)

In [147]:
odds = cm_rf.predict_proba(X_predict)

In [148]:
odds.shape

(160101, 2)

In [149]:
cm_rf.classes_

array([0, 1])

In [150]:
odds[0,:]

array([0.87620837, 0.12379163])

In [151]:
X['mortality_risk']=odds[:,1]
X['label'] = y

In [19]:
X.to_csv('processed_full_data.csv')

In [153]:
daily_df.head()

,dos,day_n,subject_id,hadm_id,icustay_id,stay_id,aniongap,albumin,bilirubin,creatinine,...,relig,marital,readmit_thirty,cirrhosis,hiv,immuno_def,hep_fail,blood_cncr,metastatic_cncr,death_4_days
0,2181-09-13,1,72707,192735,265357,72707_192735_265357,NaN,NaN,NaN,NaN,...,RELIGIOUS_NOT_SPEC,PARTNERED,0,0,0,0,0,0,0,0
1,2181-09-14,2,72707,192735,265357,72707_192735_265357,NaN,NaN,NaN,NaN,...,RELIGIOUS_NOT_SPEC,PARTNERED,0,0,0,0,0,0,0,0
2,2144-08-18,1,72555,194577,291971,72555_194577_291971,NaN,NaN,NaN,NaN,...,RELIGIOUS_NOT_SPEC,SINGLE,0,0,0,0,0,0,0,0
3,2144-08-19,2,72555,194577,291971,72555_194577_291971,12.0,NaN,NaN,1.1,...,RELIGIOUS_NOT_SPEC,SINGLE,0,0,0,0,0,0,0,0
4,2178-06-23,1,91819,184658,299557,91819_184658_299557,NaN,NaN,NaN,1.1,...,RELIGIOUS,PARTNERED,0,0,0,0,0,0,0,0


In [52]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160101 entries, 0 to 160100
Data columns (total 41 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   dos                     160101 non-null  datetime64[ns]
 1   day_n                   160101 non-null  int64         
 2   subject_id              160101 non-null  int64         
 3   hadm_id                 160101 non-null  int64         
 4   icustay_id              160101 non-null  int64         
 5   stay_id                 160101 non-null  object        
 6   aniongap                141539 non-null  float64       
 7   albumin                 75297 non-null   float64       
 8   bilirubin               81460 non-null   float64       
 9   creatinine              144507 non-null  float64       
 10  glucose                 146052 non-null  float64       
 11  hematocrit              146196 non-null  float64       
 12  hemoglobin              143972

In [53]:
daily_df['mortality_risk']=odds[:,1]

In [55]:
daily_df.to_csv('daily_datac.csv')

In [75]:
risky = np.array(daily_df.loc[daily_df['mortality_risk']>=.5, 'icustay_id'].unique())
nonrisky = np.array(daily_df.loc[daily_df['mortality_risk']<.5, 'icustay_id'].unique())

In [94]:
selection = np.append(risky[np.random.randint(0,risky.shape[0],8)],nonrisky[np.random.randint(0,nonrisky.shape[0],72)])

In [95]:
fullward = daily_df.loc[daily_df['icustay_id'].isin(selection)].copy()

Need to shift dates of service to ranges that stack all of these for last day of their stay.  Will pick a date, let's say 10/31/2018.  Then create DOS based on 10/31/2018 - (max(day_n) - day_n).

In [111]:
max_days = fullward[['icustay_id', 'day_n']].groupby('icustay_id').max()
max_days.reset_index(inplace=True)
max_days.rename({'day_n':'max_days'}, axis=1, inplace=True)

,icustay_id,max_days
0,201525,10
1,201555,15
2,201770,3
3,202467,3
4,203252,4
...,...,...
75,296667,2
76,297023,11
77,298130,2
78,298523,5


In [113]:
fullward = fullward.merge(max_days, how='inner', left_on = 'icustay_id', right_on = 'icustay_id')


In [128]:
import datetime

def newdos(srs):
    return datetime.date(2018,10,31) - datetime.timedelta(srs['max_days'] - srs['day_n'])

fullward['new_dos'] = fullward.apply(newdos, axis=1)


In [129]:
fullward.head(10)

,dos,day_n,subject_id,hadm_id,icustay_id,stay_id,aniongap,albumin,bilirubin,creatinine,...,cirrhosis,hiv,immuno_def,hep_fail,blood_cncr,metastatic_cncr,death_4_days,mortality_risk,max_days,new_dos
0,2119-05-08,1,1892,105709,203252,1892_105709_203252,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0.126512,4,2018-10-28
1,2119-05-09,2,1892,105709,203252,1892_105709_203252,17.0,NaN,NaN,2.35,...,0,0,0,0,0,0,1,0.190429,4,2018-10-29
2,2119-05-10,3,1892,105709,203252,1892_105709_203252,22.0,NaN,0.9,3.50,...,0,0,0,0,0,0,1,0.477474,4,2018-10-30
3,2119-05-11,4,1892,105709,203252,1892_105709_203252,42.0,NaN,0.9,4.20,...,0,0,0,0,0,0,1,0.476269,4,2018-10-31
4,2110-06-25,1,31489,187298,286821,31489_187298_286821,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0.058707,3,2018-10-29
5,2110-06-26,2,31489,187298,286821,31489_187298_286821,13.0,NaN,NaN,0.60,...,0,0,0,0,0,0,0,0.067158,3,2018-10-30
6,2110-06-27,3,31489,187298,286821,31489_187298_286821,13.0,3.1,NaN,0.50,...,0,0,0,0,0,0,0,0.069029,3,2018-10-31
7,2164-02-15,1,97301,195551,219603,97301_195551_219603,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0.046384,2,2018-10-30
8,2164-02-16,2,97301,195551,219603,97301_195551_219603,14.0,NaN,NaN,0.80,...,0,0,0,0,0,0,0,0.045785,2,2018-10-31
9,2169-08-16,1,4726,154344,258881,4726_154344_258881,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0.116650,2,2018-10-30


In [130]:
fullward.to_csv('fullward.csv')